In [ ]:
# Step 1 - installing the necessary libraries.
%pip install openai gradio python-dotenv

In [ ]:
# Step 2 - import
from openai import OpenAI
import gradio as gr

# for loading the api_key from .env file on vs code
import os
from dotenv import load_dotenv

# for loading the api_key from google colab
# from google.colab import userdata 

In [ ]:
# Get the API key from .env file
api_key = os.getenv("GOOGLE_API_KEY")

# Get the API key from google colab secrets
# api_key = userdata.get("GOOGLE_API_KEY")

In [5]:
# Step 3 - load base_url & api_key (https://aistudio.google.com/apikey)
client = OpenAI(
    base_url="https://generativelanguage.googleapis.com/v1beta/openai",
    api_key=api_key
)

In [6]:
# Step 4 - give a personality to our ai - Caramel AI
caramel_ai = """ You are Caramel AI – Idea Generator, built at HERE AND NOW AI – Artificial Intelligence Research Institute.

When the user requests **numbered lists**, always render in strict Markdown with exactly:

1. Top-level items numbered “1.”, “2.”, “3.”, etc.
    A. Second-level items labeled “A.”, “B.”, “C.”, etc.
        i. Third-level items labeled “i.”, “ii.”, “iii.”, etc.

Ensure there is a blank line before and after every numbered list so that Markdown renders correctly.

When the user requests **bullet lists**, always use the Unicode bullet character “•” for each item, indenting by four spaces per level:

• First-level bullet
    • Second-level bullet
        • Third-level bullet

Always leave a blank line before and after every bullet list so Markdown renders properly.

When the user requests a **paragraph**, do **not** use any list markers. Simply output one or more full sentences, without numbers, bullets, or asterisks. Ensure the text forms coherent paragraphs, with appropriate punctuation and line breaks, and respects the requested tone and length.
"""


In [7]:
# Step 5 - back-end
def get_ideas(domain, tone, length, fmt, complexity, count, user_prompt=None):
  base = (
      f"Generate {count} unique ideas in the domain of {domain}. "
      f"Write in a {tone}, {complexity} complexity, ~{length}, words each. "
      f"Format the output as a {fmt}"
  )
  if user_prompt:
    base += f"Also consider this context: {user_prompt}"

  messages = [
      {"role": "system", "content": caramel_ai},
      {"role": "user", "content": base}
  ]

  response = client.chat.completions.create(
      model="gemini-1.5-flash-latest",
      messages=messages
  )
  return response.choices[0].message.content

In [9]:
# Step 6 - front-end
logo_url = "https://raw.githubusercontent.com/hereandnowai/images/refs/heads/main/logos/HNAI%20Title%20-Teal%20%26%20Golden%20Logo%20-%20DESIGN%203%20-%20Raj-07.png"
favicon_url = "https://raw.githubusercontent.com/hereandnowai/images/refs/heads/main/logos/HNAI%20Fevicon%20-Teal%20%26%20Golden%20Logo%20-%20DESIGN%203%20-%20Raj-03.png"

favicon_head_html = f"""
<link rel="icon" type="image/png" href="{favicon_url}"/>
<link rel="shortcut icon" type="image/png" href="{favicon_url}"/>
"""

with gr.Blocks(title="Caramel AI - Idea Generator build by HERE AND NOW AI", head=favicon_head_html) as app:
  with gr.Blocks(elem_classes="logo-container"):
    gr.Image(value=logo_url, show_download_button=False, container=False, height=80)

  gr.Markdown("# Caramel AI - The Idea Generator App build by HERE AND NOW AI")

  with gr.Row():
    domain = gr.Dropdown(["Business", "Finance", "Education", "Creative Writing", "Tech"], label="Domain")
    tone = gr.Dropdown(["Professional", "Funny", "Casual", "Poetic"], label="Tone")
    length = gr.Slider(50, 1000, step=50, label="Approx. Length (words)")
  with gr.Row():
    fmt = gr.Radio(["List", "Bullets", "Paragraph"], label="Format")
    complexity = gr.Dropdown(["Beginner", "Intermediate", "Advanced"], label="Complexity")
    count = gr.Slider(1, 10, step=1, label="Number of Ideas")
  custom = gr.Textbox(lines=2, placeholder="Enter a topic for idea generation", label="What's on your mind?")

  btn = gr.Button("Generate Ideas", variant="primary")
  output = gr.Textbox(label="Ideas Generated by Caramel AI", lines=12)

  btn.click(fn=get_ideas, inputs=[domain, tone, length, fmt, complexity, count, custom], outputs=output)
  app.launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.
